# Tree-of-Thought (ToT) Prompting in Jupyter
This notebook demonstrates Tree-of-Thought reasoning with search strategies.
It runs standalone with a MockLLM but can be connected to OpenAI/OpenRouter.


In [1]:
import os, random
from typing import List, Tuple, Optional


In [2]:
class MockLLM:
    """A deterministic mock LLM for offline testing."""
    def __init__(self, seed:int=42):
        random.seed(seed)

    def generate(self, prompt:str, n:int=1) -> List[str]:
        base = "Thought based on: " + prompt.split("\n")[-1][:30]
        return [f"{base} [Option {i}]" for i in range(1, n+1)]


In [3]:
class ToTNode:
    def __init__(self, state:str, parent:Optional['ToTNode']=None, depth:int=0):
        self.state = state
        self.parent = parent
        self.depth = depth
        self.children: List[ToTNode] = []
        self.score: float = 0.0

    def path(self) -> List[str]:
        node, path = self, []
        while node:
            path.append(node.state)
            node = node.parent
        return path[::-1]


In [4]:
def tree_of_thought_search(prompt:str, llm, max_depth:int=3, beam_width:int=2):
    root = ToTNode(prompt, parent=None, depth=0)
    frontier = [root]

    for depth in range(max_depth):
        new_frontier = []
        for node in frontier:
            thoughts = llm.generate(node.state, n=beam_width)
            for t in thoughts:
                child = ToTNode(t, parent=node, depth=depth+1)
                child.score = random.random()  # heuristic scoring
                node.children.append(child)
                new_frontier.append(child)
        frontier = sorted(new_frontier, key=lambda x: -x.score)[:beam_width]

    return frontier


In [5]:
llm = MockLLM()
final_nodes = tree_of_thought_search("How can we reduce traffic in cities?", llm)

for node in final_nodes:
    print("Path:", " -> ".join(node.path()))
    print("Score:", node.score, "\n")


Path: How can we reduce traffic in cities? -> Thought based on: How can we reduce traffic in c [Option 2] -> Thought based on: Thought based on: How can we r [Option 1] -> Thought based on: Thought based on: Thought base [Option 1]
Score: 0.8921795677048454 

Path: How can we reduce traffic in cities? -> Thought based on: How can we reduce traffic in c [Option 2] -> Thought based on: Thought based on: How can we r [Option 2] -> Thought based on: Thought based on: Thought base [Option 1]
Score: 0.4219218196852704 

